# Example 3 Code

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Play, VBox, HBox, jslink
from IPython.display import display
from typing import Tuple

# Konstanta Fisika
G = 6.67430e-11
M_sun = 1.989e30
M_earth = 5.972e24
M_moon = 7.342e22
AU = 1.496e11
MOON_DIST = 384.4e6
EARTH_VEL = 29_780
MOON_VEL = 1022

# Parameter waktu
dt = 900        # Waktu per langkah: 15 menit (dalam detik)
total_days = 120
steps = int((total_days * 24 * 3600) / dt)

# State Vector: [x, y, vx, vy] untuk Sun, Earth, Moon
state = np.array([
    0.0, 0.0, 0.0, 0.0,                      # Sun
    AU, 0.0, 0.0, EARTH_VEL,                  # Earth
    AU + MOON_DIST, 0.0, 0.0, EARTH_VEL + MOON_VEL  # Moon
], dtype='float64')

def grav_acc(x1: float, y1: float, x2: float, y2: float, m2: float) -> Tuple[float, float]:
    dx = x2 - x1
    dy = y2 - y1
    r = np.hypot(dx, dy) + 1e-5  # Hindari div by 0
    r3 = r**3
    return G * m2 * dx / r3, G * m2 * dy / r3

def derivatives(s: np.ndarray) -> np.ndarray:
    x_s, y_s, vx_s, vy_s = s[0:4]
    x_e, y_e, vx_e, vy_e = s[4:8]
    x_m, y_m, vx_m, vy_m = s[8:12]

    # Sun affected by Earth and Moon
    a_s_e = grav_acc(x_s, y_s, x_e, y_e, M_earth)
    a_s_m = grav_acc(x_s, y_s, x_m, y_m, M_moon)
    ax_s = a_s_e[0] + a_s_m[0]
    ay_s = a_s_e[1] + a_s_m[1]

    # Earth affected by Sun and Moon
    a_e_s = grav_acc(x_e, y_e, x_s, y_s, M_sun)
    a_e_m = grav_acc(x_e, y_e, x_m, y_m, M_moon)
    ax_e = a_e_s[0] + a_e_m[0]
    ay_e = a_e_s[1] + a_e_m[1]

    # Moon affected by Sun and Earth
    a_m_s = grav_acc(x_m, y_m, x_s, y_s, M_sun)
    a_m_e = grav_acc(x_m, y_m, x_e, y_e, M_earth)
    ax_m = a_m_s[0] + a_m_e[0]
    ay_m = a_m_s[1] + a_m_e[1]

    return np.array([
        vx_s, vy_s, ax_s, ay_s,
        vx_e, vy_e, ax_e, ay_e,
        vx_m, vy_m, ax_m, ay_m
    ])

def rk4_steps(s: np.ndarray, dt: float) -> np.ndarray:
    k1 = derivatives(s)
    k2 = derivatives(s + 0.5 * dt * k1)
    k3 = derivatives(s + 0.5 * dt * k2)
    k4 = derivatives(s + dt * k3)
    return s + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

# Simulasi RK4
states = []
temp = state.copy()
for _ in range(steps):
    states.append(temp.copy())
    temp = rk4_steps(temp, dt)

states = np.array(states)

# Debug: Cek posisi awal dan akhir
print("Posisi awal Bumi:", states[0][4:6])
print("Posisi akhir Bumi:", states[-1][4:6])
print("Posisi awal Bulan:", states[0][8:10])
print("Posisi akhir Bulan:", states[-1][8:10])

# Fungsi visualisasi
def plot_orbit(i: int, zoom: float = 2.0) -> None:
    fig, ax = plt.subplots(figsize=(7, 7))
    fig.set_facecolor("black") # <-- THIS LINE WAS ADDED
    ax.set_facecolor("black")

    # Jejak orbit
    if i > 0:
        earth_x = []
        earth_y = []
        moon_x = []
        moon_y = []
        for j in range(i):
            earth_x.append(states[j][4])
            earth_y.append(states[j][5])
            moon_x.append(states[j][8])
            moon_y.append(states[j][9])
        ax.plot(earth_x, earth_y, 'b-', lw=0.8, label='Orbit Bumi')
        ax.plot(moon_x, moon_y, 'gray', lw=0.5, label='Orbit Bulan')

    # Objek saat ini
    current_state = states[i]
    ax.plot(current_state[0], current_state[1], 'yo', markersize=8, label='Matahari')
    ax.plot(current_state[4], current_state[5], 'bo', markersize=6, label='Bumi')
    ax.plot(current_state[8], current_state[9], 'wo', markersize=3, label='Bulan')

    ax.set_xlim(-zoom * AU, zoom * AU)
    ax.set_ylim(-zoom * AU, zoom * AU)
    ax.set_aspect('equal')
    ax.set_title(f'Hari ke-{int(i * dt // (24 * 3600))}', color='white')
    ax.tick_params(colors='white')
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    ax.legend(facecolor='black', labelcolor='white', loc='upper right')
    plt.show()

# Widget Interaktif
play = Play(min=0, max=len(states) - 1, step=1, interval=20, description='▶')
slider = IntSlider(min=0, max=len(states) - 1, step=1, description='Frame')
zoom_slider = FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='Zoom')

# Sinkronisasi play dan slider
jslink((play, 'value'), (slider, 'value'))

controls = HBox([play, slider])
interact_ui = interact(plot_orbit, i=slider, zoom=zoom_slider)
display(VBox([controls, zoom_slider]))

Posisi awal Bumi: [1.496e+11 0.000e+00]
Posisi akhir Bumi: [-7.08381629e+10  1.31833627e+11]
Posisi awal Bulan: [1.499844e+11 0.000000e+00]
Posisi akhir Bulan: [-7.12065106e+10  1.31906845e+11]


interactive(children=(IntSlider(value=0, description='Frame', max=11519), FloatSlider(value=2.0, description='…

# Example 4 Code

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Play, FloatSlider, VBox, HBox, jslink
from IPython.display import display
from typing import Tuple

# Initial Parameters
G = 6.67430e-11  # Gravitational constant
M = 6e24  # Mass of all planets (same)

# Initial positions (intentionally asymmetric)
r1 = np.array([0.0, 0.0])
r2 = np.array([1.0e11, 0.0])
r3 = np.array([0.5e11, 0.87e11])

# Initial velocities (different but close)
v1 = np.array([0.0, 12000])
v2 = np.array([0.0, -11000])
v3 = np.array([12000, 0.0])

positions = np.array([r1, r2, r3])
velocities = np.array([v1, v2, v3])
masses = np.array([M, M, M])

# Gravitational force function
def get_acc(pos: np.ndarray, masses: np.ndarray) -> np.ndarray:
    n = len(masses)
    acc = np.zeros_like(pos)
    for i in range(n):
        for j in range(n):
            if i != j:
                r = pos[j] - pos[i]
                dist = np.linalg.norm(r) + 1e-5
                acc[i] += G * masses[j] * r / dist**3
    return acc

# RK4 Integrator
def rk4_integrator(pos: np.ndarray, vel: np.ndarray, dt: float) -> Tuple[np.ndarray, np.ndarray]:
    def acc(p): return get_acc(p, masses)

    a1 = acc(pos)
    v1 = vel

    a2 = acc(pos + 0.5 * dt * v1)
    v2 = vel + 0.5 * dt * a1

    a3 = acc(pos + 0.5 * dt * v2)
    v3 = vel + 0.5 * dt * a2

    a4 = acc(pos + dt * v3)
    v4 = vel + dt * a3

    pos_new = pos + (dt / 6.0) * (v1 + 2 * v2 + 2 * v3 + v4)
    vel_new = vel + (dt / 6.0) * (a1 + 2 * a2 + 2 * a3 + a4)

    return pos_new, vel_new

# Simulation
dt = 3000    # 50 minutes
steps = 3000

pos_hist = []
vel_hist = []

p, v = positions.copy(), velocities.copy()
for _ in range(steps):
    pos_hist.append(p.copy())
    vel_hist.append(v.copy())
    p, v = rk4_integrator(p, v, dt)

pos_hist = np.array(pos_hist)

print(f"Simulation completed: {steps} steps")
print(f"Time step: {dt/60:.1f} minutes")
print(f"Total simulation time: {(steps * dt) / 86400:.1f} days")

# Plot Function
def plot_system(frame: int = 0, zoom: float = 1.5) -> None:
    fig, ax = plt.subplots(figsize=(7, 7))
    fig.set_facecolor('black') # <-- THIS LINE WAS ADDED
    ax.set_facecolor('black')

    colors = ['cyan', 'magenta', 'yellow']
    labels = ['Planet 1', 'Planet 2', 'Planet 3']

    for i in range(3):
        if frame > 1:
            # Plot orbital trails
            trail_x = [pos_hist[j][i][0] for j in range(frame)]
            trail_y = [pos_hist[j][i][1] for j in range(frame)]
            ax.plot(trail_x, trail_y, '-', color=colors[i], alpha=0.7, linewidth=1, label=labels[i])

        # Plot current position
        current_pos = pos_hist[frame][i]
        ax.plot(current_pos[0], current_pos[1], 'o', color=colors[i], markersize=8,
                label=labels[i] if frame <= 1 else "")

    ax.set_xlim(-zoom*1e11, zoom*1e11)
    ax.set_ylim(-zoom*1e11, zoom*1e11)
    ax.set_aspect('equal')
    ax.set_title(f'Three Planet Chaos | Day {(frame * dt) // 86400}', color='white')
    ax.tick_params(colors='white')
    ax.set_xlabel('Distance (m)', color='white')
    ax.set_ylabel('Distance (m)', color='white')
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    if frame > 0:
        # This prevents multiple labels from appearing for the same planet
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(), facecolor='black', labelcolor='white', loc='upper right')
    ax.grid(True, color='gray', alpha=0.3, linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

# GUI Widgets
play = Play(min=0, max=steps-1, step=1, interval=50, description='Play')
slider = IntSlider(min=0, max=steps-1, step=1, description='Frame')
zoom_slider = FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description='Zoom')

# Link play button with frame slider
jslink((play, 'value'), (slider, 'value'))

# Create control panel layout
controls = HBox([play, slider])
print("\n=== Interactive Three-Body Chaos Simulation ===")
print("Use the controls below to explore the chaotic motion:")
display(VBox([controls, zoom_slider]))
interact(plot_system, frame=slider, zoom=zoom_slider)

Simulation completed: 3000 steps
Time step: 50.0 minutes
Total simulation time: 104.2 days

=== Interactive Three-Body Chaos Simulation ===
Use the controls below to explore the chaotic motion:


interactive(children=(IntSlider(value=0, description='Frame', max=2999), FloatSlider(value=1.5, description='Z…

<function __main__.plot_system(frame: int = 0, zoom: float = 1.5) -> None>